In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

#Read data from chunks csv file
df = pd.read_csv('/content/drive/MyDrive/INLPT_Project/csv_files/chunks.csv')

In [3]:
%pip install python-dotenv

In [4]:
import os
from dotenv import load_dotenv

# load environment variables from .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [31]:
%pip install pinecone-client

In [10]:
!pip install spacy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 32.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [12]:
import pandas as pd
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_md")

# Function to get spaCy embedding for a text string
def get_embedding(text):
    doc = nlp(text)
    # Use the vector for the entire document
    return doc.vector

# Apply the function to the 'text' column and create a new 'embedding' column
df['embedding'] = df['text'].apply(get_embedding)

In [13]:
# Display the DataFrame with embeddings
df

,Unnamed: 0,id,CELEX number,text,extras,embedding
0,0,0,21975A1201(01),Avis juridique important Cooperation Agreement...,NaN,"[-1.9581965, -3.8455784, 1.5960116, 0.6923731,..."
1,1,1,21975A1201(01),EUROPEAN ATOMIC ENERGY COMMUNITY AND THE INTER...,NaN,"[-2.6876938, -0.8575195, 0.42969406, 0.2007098..."
2,2,2,21975A1201(01),Contracting Parties shall consult each other r...,NaN,"[-2.7651699, 0.11774456, -2.2513375, 0.8061124..."
3,3,3,21975A1201(01),with respect to items on their agenda in which...,NaN,"[-2.2858288, -0.975391, -1.1897662, 0.509713, ..."
4,4,4,21975A1201(01),confidential nature of certain information and...,NaN,"[-2.9722536, -0.85482717, -0.27737385, 0.74818..."
...,...,...,...,...,...,...
25181,25181,25181,32023R2633,ELI: http://data.europa.eu/eli/reg_impl/2023/2...,Section: 'Document',"[-1.9006834, -3.113775, 4.7329025, 1.2058558, ..."
25182,25182,25182,42009D0913,DECISION TAKEN BY COMMON AGREEMENT BETWEEN THE...,"Section: 'Document'; Section: 'Document', (1)","[-2.9382653, -3.208069, 1.6150558, 0.3734278, ..."
25183,25183,25183,42009D0913,The location of the seat of this Agency should...,"Section: 'Document', (2); Section: 'Document';...","[-2.7157636, -1.5171125, 0.0045820503, 1.23015..."
25184,25184,25184,42010D0349,DECISION TAKEN BY COMMON ACCORD BETWEEN THE RE...,"Section: 'Document', (2); Section: 'Document';...","[-3.2186131, -3.001123, 1.5925204, 0.339502, 3..."


In [14]:
import numpy as np

# Find the maximum length of embeddings
max_length = df['embedding'].apply(len).max()
print(max_length)

# Pad or truncate embeddings to a fixed size
df['embedding'] = df['embedding'].apply(lambda emb: np.pad(emb, (0, max_length - len(emb)), 'constant'))

# Convert the list of embeddings to a 2D numpy array
embeddings_array = np.vstack(df['embedding'].to_numpy())

# Check if there are valid embeddings
if embeddings_array.shape[0] == 0:
    print("No valid embeddings found.")
else:
    # Print the embeddings
    print(embeddings_array)
    print(embeddings_array.shape)

300
[[-1.9581965  -3.8455784   1.5960116  ... -2.022801    0.26372543
  -0.68839526]
 [-2.6876938  -0.8575195   0.42969406 ... -2.697899   -1.37923
   0.8247703 ]
 [-2.7651699   0.11774456 -2.2513375  ... -3.4232373  -2.876724
   0.82123375]
 ...
 [-2.7157636  -1.5171125   0.00458205 ... -3.2380555  -0.66828096
  -0.34325856]
 [-3.2186131  -3.001123    1.5925204  ... -3.2356331  -0.01843528
  -0.23140563]
 [-2.8043635  -1.7736311   0.49761322 ... -3.4420178  -0.20802231
  -0.66664046]]
(25186, 300)


In [15]:
import os
from pinecone import Pinecone, ServerlessSpec

# Get Pinecone API key and environment from environment variables
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

# Create an instance of the Pinecone class
pc = Pinecone(api_key=pinecone_api_key)


In [24]:
index_name = 'inlpt-project'

if index_name not in pc.list_indexes().names():
  pc.create_index(
    name=index_name,
    dimension=len(embeddings_array[0]),
    metric='cosine',
    spec=ServerlessSpec(
      cloud="aws",
      region="us-central1"
    )
  )

In [25]:
index_name = 'inlpt-project'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 300,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [29]:
batch_size = 32

for i in range(0, len(df), batch_size):
    i_end = min(len(df), i+batch_size)
    batch = df.iloc[i:i_end]
    ids = [f"{x['id']}" for i, x in batch.iterrows()]
    texts = [x['text'] for i, x in batch.iterrows()]
    embeds = df['embedding']
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['text'],
         'CELEX number': x['CELEX number']} for i, x in batch.iterrows()
    ]
    index.upsert(vectors=zip(ids, embeds, metadata))



In [30]:
index.describe_index_stats()

{'dimension': 300,
 'index_fullness': 0.25186,
 'namespaces': {'': {'vector_count': 25186}},
 'total_vector_count': 25186}